In [2]:
import os
import numpy as np
import cantera as ct

# load Tamaoki mechanism

In [ ]:
#create gas from original mechanism file gri30.cti
gas = ct.Solution('NH3_Tamaoki.yaml')
#reorder the gas to match pyJac
n2_ind = gas.species_index('N2')
specs = gas.species()[:]
gas = ct.Solution(thermo='IdealGas', kinetics='GasKinetics',
        species=specs[:n2_ind] + specs[n2_ind + 1:] + [specs[n2_ind]],
        reactions=gas.reactions())

/tmp/ipykernel_78055/3612279425.py:6: DeprecationWarning: FactoryBase::canonicalize: Model name 'IdealGas' is deprecated. Use 'ideal-gas' instead.
  gas = ct.Solution(thermo='IdealGas', kinetics='GasKinetics',


In [12]:
for item in gas.species_names:

    print(item)

HE
N2
H2
O2
H
O
OH
HO2
H2O
H2O2
OH*
N
NH3
NH2
NH
NNH
NO
N2O
HNO
HON
H2NO
HNOH
NH2OH
NO2
HONO
HNO2
NO3
HONO2
N2H2
H2NN
N2H4
N2H3
AR


In [4]:
stoichi_coeffs_ct = gas.product_stoich_coeffs - gas.reactant_stoich_coeffs

# load spec_rates.c

In [5]:
path_c = '../src/c/spec_rates.c'
print(os.path.exists(path_c))

True


In [6]:
stoich_coeffs_cema = np.zeros(shape=(gas.n_species, gas.n_reactions), dtype=np.int32)
list_k_pres_mod = np.zeros(shape=gas.n_reactions, dtype=np.int32)
list_i_rev_rates = np.zeros(shape=gas.n_reactions, dtype=np.int32)

with open(path_c, 'r', encoding='utf-8') as file:
        lines = file.readlines()

for line in lines:

      # pick reaction number
      if '//rxn' in line:
            i_reaction = int(line.split(' ')[-1])
            # print('reaction ', i_reaction)

      # pick species number
      if '//sp' in line:
            j_species = int(line.split(' ')[-1]) -1
            # print('species ', j_species)

      # pick coefficient
      coeff = 1.0
      
      if 'sp_rates' in line and '=' in line:

            if 'sp_rates[0]' in line:
                  continue
    
            sign_equals = line.split('=')[0][-1:]
            sign_bracket = line.split('(')[0][-1:]

            if sign_equals == '-':
                  coeff *= - 1.0
                  
            if sign_bracket == '-':
                  coeff *= - 1.0

            if '*' in line.split('=')[1].split('fwd_rates')[0]:
                  # print(float(line.split('=')[1].split('fwd_rates')[0].split('*')[0]))
                  coeff *= float(line.split('=')[1].split('fwd_rates')[0].split('*')[0])

            # print(coeff, line)
            stoich_coeffs_cema[j_species, i_reaction] = coeff

            if 'rev_rates' in line:
                  i_rev_rates = int(line.split('rev_rates[')[1].split(']')[0])
                  # print(i_rev_rates, line)
                  list_i_rev_rates[i_reaction] = i_rev_rates
            else:
                  list_i_rev_rates[i_reaction] = -1
                  
            
      
            if 'pres_mod' in line:
                  k_pres_mod = int(line.split('* pres_mod')[-1].split('[')[1].split(']')[0])
                  # print(k_pres_mod, i_reaction, line)
                  list_k_pres_mod[i_reaction] = k_pres_mod
            else:
                  list_k_pres_mod[i_reaction] = -1


## Export indexes

In [7]:
stoich_coeffs_cema.reshape([gas.n_species*gas.n_reactions],order='F').tofile("stoich_coeffs.bin")
list_i_rev_rates.tofile("list_i_rev_rates.bin")
list_k_pres_mod.tofile("list_k_pres_mod.bin")


In [8]:
stoich_coeffs_cema[:, -1]

array([ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [9]:
list_i_rev_rates

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [10]:
list_k_pres_mod

array([-1, -1, -1, -1, -1, -1,  0, -1, -1,  1, -1, -1,  2,  3, -1,  4, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  5, -1, -1, -1, -1, -1,
        6, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1,  7, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  9, 10, 11,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, 12, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, 13, -1, -1, -1, -1, -1, -1, -1, 14, 15, -1, -1, -1, -1,
       -1, -1, -1, 16, -1, -1, -1, -1, -1, -1, -1, -1, 17, -1, -1, -1, 18,
       -1, -1, -1, -1, -1, 19, -1, -1, -1, -1, 20, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, 21, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 22, 23, 24, -1,
       -1, -1, -1, -1, -1